INSTALLING THE PROPHET

 Prophet will be our baseline model and we need to prove that AI adds value. Facebook prophet is the industry standard for time-series forecasting. It looks at trends and holidays, ignoring sentiment entirely. 

 We will get a MAE score and if the LSTM beats this score later, we have a winning project.
